In [1]:
from custom.model import FreeConvNetwork
from torchsummary import summary
import torch

# Testing & summarizeing the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FreeConvNetwork().to(device)

summary(model, (3, 218, 178))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
LocallyConnected2d-1            [-1, 8, 54, 44]       2,071,872
              ReLU-2            [-1, 8, 54, 44]               0
LocallyConnected2d-3           [-1, 16, 13, 10]         601,120
              ReLU-4           [-1, 16, 13, 10]               0
         MaxPool2d-5             [-1, 16, 6, 5]               0
           Flatten-6                  [-1, 480]               0
            Linear-7                  [-1, 300]         144,300
              ReLU-8                  [-1, 300]               0
            Linear-9                  [-1, 200]          60,200
Total params: 2,877,492
Trainable params: 2,877,492
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.44
Forward/backward pass size (MB): 0.34
Params size (MB): 10.98
Estimated Total Size (MB): 11.76
-------------------------------------

In [3]:
from utils.create_train_test_folder_structure import create_train_test_folder_structure
import os
create_train_test_folder_structure(100, os.getcwd())


File Not Found: 161375.jpg
File Not Found: 159184.jpg
File Not Found: 159371.jpg
File Not Found: 162245.jpg
File Not Found: 160601.jpg
File Not Found: 160531.jpg
File Not Found: 161392.jpg
File Not Found: 161396.jpg
File Not Found: 161220.jpg
File Not Found: 159537.jpg
File Not Found: 162563.jpg
File Not Found: 162365.jpg
File Not Found: 159421.jpg
File Not Found: 159497.jpg
File Not Found: 159710.jpg
File Not Found: 160247.jpg
File Not Found: 160679.jpg
File Not Found: 161347.jpg
File Not Found: 159948.jpg
File Not Found: 162676.jpg
File Not Found: 161619.jpg
File Not Found: 161827.jpg
File Not Found: 160861.jpg
File Not Found: 162217.jpg
File Not Found: 160831.jpg
File Not Found: 162477.jpg
File Not Found: 160421.jpg
File Not Found: 160850.jpg
File Not Found: 161473.jpg
File Not Found: 160370.jpg
File Not Found: 159789.jpg
File Not Found: 162248.jpg
File Not Found: 159588.jpg
File Not Found: 160570.jpg
File Not Found: 159190.jpg
File Not Found: 160298.jpg
File Not Found: 160355.jpg
F

In [1]:
from torchvision import datasets,transforms
from sklearn.model_selection import StratifiedShuffleSplit
import torch

transform = transforms.Compose(
    [transforms.ToTensor()]
)

dataset = datasets.ImageFolder("./Data/img_align_celeba/class_data/", transform=transform)

#list all the labels in assigned to the dataset by the ImageFolder method
labels = [label for _, label in dataset]

#make a train/test split from based on the labels using StratifiedShuffleSplit to make sure all labels are evenly represented
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_indices, test_indices = next(splitter.split(dataset, labels))

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

#define the dataset loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=50)

#report split sizes
print("Training set size: {}".format(len(train_dataset)))
print("Testing set size: {}".format(len(test_dataset)))

Training set size: 3918
Testing set size: 980


In [5]:
for inputs, labels in train_loader:
    print(labels)

tensor([14, 52, 99, 24])
tensor([116, 166, 163,  38])
tensor([193,  93, 153, 142])
tensor([116,  75,  42, 107])
tensor([  7, 110, 112, 178])
tensor([186, 187,  29, 164])
tensor([163,  78, 197, 176])
tensor([156,  57, 109, 103])
tensor([ 42,  59, 105, 120])
tensor([ 55, 140,  55, 152])
tensor([102,  73, 127, 191])
tensor([189,  36, 143,   7])
tensor([14, 23, 75, 68])
tensor([130, 153, 125, 182])
tensor([154,  89, 137, 192])
tensor([80, 67, 55, 72])
tensor([138, 140, 187,  60])
tensor([ 28, 159, 102, 196])
tensor([ 45,  35, 144, 125])
tensor([ 33, 147, 194,  21])
tensor([ 78, 144, 172, 154])
tensor([65, 28, 31, 17])
tensor([ 35,  13,  72, 110])
tensor([183, 161, 166,  84])
tensor([150, 133, 173,  41])
tensor([123, 116, 104,  11])
tensor([131, 164, 123, 127])
tensor([ 37, 166,  53, 186])
tensor([47, 38, 17, 63])
tensor([  7,   9,  78, 181])
tensor([ 40, 147, 124, 116])
tensor([ 94,  63,  57, 143])
tensor([133,  32, 100, 115])
tensor([ 99,   3, 126, 141])
tensor([ 80, 109,  49,   5])
tenso

In [2]:
import torch.nn as nn
import torch
from custom.model import FreeConvNetwork
import torch

#moving model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FreeConvNetwork().to(device)

#defining the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [3]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # moving the training data batch by batch to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.


for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(test_loader):
        vinputs, vlabels = vdata
         
        # moving the test data batch by batch to the GPU
        vinputs, vlabels = vinputs.to(device), vlabels.to(device)
        
        # validating the model
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss.detach() # wihtout .detach I get an out of memory error on GPU

        #remove data from GPU
        #del vinputs, vlabels, voutputs, vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
LOSS train 0.0 valid 5.300866603851318
EPOCH 2:
LOSS train 0.0 valid 5.301135063171387
EPOCH 3:
LOSS train 0.0 valid 5.3018717765808105
EPOCH 4:
LOSS train 0.0 valid 5.3018717765808105
EPOCH 5:
LOSS train 0.0 valid 5.3018717765808105


In [8]:
enumerate(train_loader)